# Homework 3

I have a catalog of repeating earthquakes, and I want to download seismic waveforms corresponding to these repeating earthquakes. However, when I look at the list of stations available in the seismic network, there are more than 6000. I do not want to download data from 6000 stations, so I want to filter only the seismic stations that are relevant for what I want to do with my waveforms.

In [1]:
# Address of the website to download data
url = 'http://ncedc.org/ftp/pub/doc/NC.info/NC.channel.summary.day'

In [2]:
# Useful Python modules
import numpy as np
import pandas as pd
import io
import pickle
import requests
from datetime import datetime, timedelta
from math import cos, sin, pi, sqrt

In [3]:
# Import the data from the website into a Python dataframe
s = requests.get(url).content
data = pd.read_csv(io.StringIO(s.decode('utf-8')), header=None, skiprows=2, sep='\s+', usecols=list(range(0, 13)))
data.columns = ['station', 'network', 'channel', 'location', 'rate', 'start_time', 'end_time', 'latitude', 'longitude', 'elevation', 'depth', 'dip', 'azimuth']

In [4]:
# Transform columns start_time and end_time into datetime format
startdate = pd.to_datetime(data['start_time'], format='%Y/%m/%d,%H:%M:%S')
data['start_time'] = startdate
# Avoid 'OutOfBoundsDatetime' error with year 3000
enddate = data['end_time'].str.replace('3000', '2025')
enddate = pd.to_datetime(enddate, format='%Y/%m/%d,%H:%M:%S')
data['end_time'] = enddate

After discussing with my adviser, we decided than only the following channels are relevant for the work we want to do:

In [5]:
channels = ['BHE', 'BHN', 'BHZ', 'BH1', 'BH2', \
            'EHE', 'EHN', 'EHZ', 'EH1', 'EH2', \
            'HHE', 'HHN', 'HHZ', 'HH1', 'HH2', \
            'SHE', 'SHN', 'SHZ', 'SH1', 'SH2']

## First question

Filter the dataset to keep only the rows with the channels as defined above.

In [6]:
data = data.loc[data['channel'].isin(channels)]

My earthquake catalog starts on 2007/07/01 and ends on 2009/07/01. I am only interested in stations that started recording before 2007/07/01 and ended recording after 2009/07/01.

## Second question

Filter the dataset to keep only stations that started recording before 2007/07/01 and ended recording after 2009/07/01.

In [16]:
startDate = '2007/07/01'
endDate = '2009/07/01'

timeSpan = data.groupby(['station']).agg({'start_time':lambda x: min(x), 'end_time':lambda x: max(x)})
mask = (timeSpan.start_time < startDate) & (timeSpan.end_time > endDate)
timeSpan = timeSpan[mask]
data = data.loc[data.station.isin(timeSpan.index.tolist())]

I only want to keep the stations that are located less than 100 km from my repeating earthquakes. For stations farther away, the signal-to-noise ratio would be too low.

The earthquakes are located at latitude = 40.09 and longitude = -122.87. Here is a function to compute the distance from the station to the earthquakes, and to add a column distance to the dataset

In [17]:
a = 6378.136
e = 0.006694470
lat0 = 40.09000
lon0 = -122.87000
dx = (pi / 180.0) * a * cos(lat0 * pi / 180.0) / sqrt(1.0 - e * e * sin(lat0 * pi / 180.0) * sin(lat0 * pi / 180.0))
dy = (3.6 * pi / 648.0) * a * (1.0 - e * e) / ((1.0 - e * e * sin(lat0 * pi / 180.0) * sin(lat0 * pi / 180.0)) ** 1.5)
x = dx * (data['longitude'] - lon0)
y = dy * (data['latitude'] - lat0)
data['distance'] = np.sqrt(np.power(x, 2.0) + np.power(y, 2.0))

## Third question

Filter the dataset to keep only stations that are less than 100 km from the earthquakes.

In [18]:
data = data.loc[data['distance'] < 100]

Finally, I want to group the result such that the final result looks like:

|station|network|location|latitude|longitude |elevation|depth|distance |channel    |start_time         |end_time           |
|-------|-------|--------|--------|----------|---------|-----|---------|-----------|-------------------|-------------------|
|KBS 	|NC 	|-- 	 |39.91719|-123.59561|1120.0   |0.0  |64.720762|SHZ        |2002-10-17 00:00:00|2011-10-27 21:25:00|
|KCPB 	|NC 	|-- 	 |39.68631|-123.58242|1261.0   |0.0  |75.502041|HHZ,HHN,HHE|2006-10-18 00:08:00|2010-11-01 22:00:00|

I want all different channels to be grouped together, instead of having one row per channel. I also want to get the start_time end end_time for each station, instead of having it for each channel.

You can use the following function to group the channels together:

In [19]:
def f(x):
    """
    Concatenate channels
    """
    result = '%s' % ','.join(x)
    result = list(set(result.split(',')))
    result = '%s' % ','.join(result)
    return result

## Fourth question

Use the pandas function agg to group the channels of a given station together, and compute the least recent start_time and the most recent end_time for each station.

In [21]:
final = data.groupby(['station','network','latitude','elevation','depth']).agg({'channel': f,'start_time':lambda x: min(x), 'end_time':lambda x: max(x)})
final.reset_index()

,station,latitude,channel,start_time,end_time
0,GBB,39.80127,"SHZ,EHZ",2000-12-06 18:38:00,2025-01-01 00:00:00
1,GCK,39.54375,"SHZ,EHZ",2000-06-06 21:58:00,2025-01-01 00:00:00
2,GFC,39.32654,EHZ,2020-06-23 20:27:00,2025-01-01 00:00:00
3,GFC,39.32655,"SHZ,EHZ",2001-04-03 23:25:00,2020-03-18 22:53:00
4,GHM,39.49545,"SHZ,EHZ",1984-01-01 00:00:00,2025-01-01 00:00:00
5,GRO,39.91684,"SHZ,EHZ",1990-12-13 23:30:00,2025-01-01 00:00:00
6,GTC,39.39944,"SHZ,EHZ",1996-08-01 22:08:00,2025-01-01 00:00:00
7,GTS,39.31161,"SHZ,EHZ",1984-01-01 00:00:00,2020-10-19 16:57:00
8,GVV,39.77510,"SHZ,EHZ",2002-04-28 00:00:00,2021-03-09 00:11:00
9,KBN,39.89237,"SHZ,EHZ",1984-01-01 00:00:00,2025-01-01 00:00:00


## Fifth question

How many stations are left in the dataset?

***There are 29 station left.***